<a href="https://colab.research.google.com/github/Emma-Coco/Screw-project/blob/main/Modele_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!unzip screw_dataset.zip


Archive:  screw_dataset.zip
   creating: screw_dataset/
  inflating: __MACOSX/._screw_dataset  
  inflating: screw_dataset/.DS_Store  
  inflating: __MACOSX/screw_dataset/._.DS_Store  
   creating: screw_dataset/bad/
  inflating: __MACOSX/screw_dataset/._bad  
   creating: screw_dataset/good/
  inflating: __MACOSX/screw_dataset/._good  
  inflating: screw_dataset/bad/023_png.rf.53e5fc79e243548f8792354dae3acf09.jpg  
  inflating: __MACOSX/screw_dataset/bad/._023_png.rf.53e5fc79e243548f8792354dae3acf09.jpg  
  inflating: screw_dataset/bad/006_png.rf.dba2300aaf35998d723c334bd8d334f7.jpg  
  inflating: __MACOSX/screw_dataset/bad/._006_png.rf.dba2300aaf35998d723c334bd8d334f7.jpg  
  inflating: screw_dataset/bad/013_png.rf.54ba3ab35a37b312543c98d486034db1.jpg  
  inflating: __MACOSX/screw_dataset/bad/._013_png.rf.54ba3ab35a37b312543c98d486034db1.jpg  
  inflating: screw_dataset/bad/001_png.rf.a9c493c563488514026a6f9dee8e857c.jpg  
  inflating: __MACOSX/screw_dataset/bad/._001_png.rf.a9c493c5

In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, precision_recall_curve, confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pathlib
import os
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Configuration
DATA_DIR = "./screw_dataset"
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
SEED = 42
TEST_RATIO = 0.15
VAL_RATIO = 0.15
MODEL_SAVE_PATH = 'mon_cnn.h5'
INITIAL_LR = 0.002  # Learning rate augmenté (était 0.001 par défaut)

# --------------------------------------------------
# Les autres fonctions restent identiques...
# --------------------------------------------------

# --------------------------------------------------
# Étape 6: Création et entraînement du modèle (Learning rate augmenté)
# --------------------------------------------------
def create_and_train_model():
    """
    Crée et entraîne le modèle CNN basé sur MobileNetV2.

    Returns:
        tuple: (model, history, test_dataset)
    """
    # Obtenir les datasets et le nombre de steps
    (train_dataset, val_dataset, test_dataset, steps_per_epoch, validation_steps,
     X_train, X_val, X_test, y_train, y_val, y_test, classes) = data_pipeline()

    # Créer le modèle MobileNetV2
    base_model = MobileNetV2(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )

    # Geler les couches de base_model
    base_model.trainable = False

    # Ajouter des couches personnalisées
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    # Créer le modèle final
    model = Model(inputs=base_model.input, outputs=predictions)

    # Utiliser un optimiseur Adam avec un learning rate plus élevé
    optimizer = Adam(learning_rate=INITIAL_LR)

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Recall(name='recall')]
    )

    # Afficher le résumé du modèle
    model.summary()

    # Callbacks
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1,
        mode='min'
    )

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        mode='min',
        restore_best_weights=True
    )

    checkpoint = ModelCheckpoint(
        MODEL_SAVE_PATH,
        save_best_only=True,
        monitor='val_loss',
        mode='min'
    )

    # Entraînement du modèle
    print("\n" + "="*50)
    print("Démarrage de l'entraînement...")

    history = model.fit(
        train_dataset,
        epochs=50,
        steps_per_epoch=steps_per_epoch,
        validation_data=val_dataset,
        validation_steps=validation_steps,
        callbacks=[early_stopping, checkpoint, reduce_lr]
    )

    # Évaluation du modèle sur le dataset de test
    print("\n" + "="*50)
    print("Évaluation du modèle sur le test set...")
    test_results = model.evaluate(test_dataset)

    # Afficher les résultats
    metric_names = ['loss', 'accuracy', 'recall']
    for name, value in zip(metric_names, test_results):
        print(f"{name}: {value:.4f}")

    # Visualiser les courbes d'apprentissage
    plot_training_history(history)

    return model, history, test_dataset

# --------------------------------------------------
# Étape 8: Génération de la courbe ROC avec seuil personnalisé
# --------------------------------------------------
def generate_roc_curve_evaluation(min_recall=0.90):
    """
    Fonction pour évaluer le modèle et générer les courbes ROC.
    Permet de choisir un seuil qui garantit un rappel minimum (minimiser les faux négatifs).

    Args:
        min_recall: Rappel minimum souhaité (défaut=0.90)

    Returns:
        tuple: (roc_auc, pr_auc, optimal_threshold, optimal_custom_threshold)
    """
    print("="*50)
    print(f"Évaluation du modèle avec courbe ROC (rappel minimum: {min_recall:.2f})")
    print("="*50)

    # 1. Charger le modèle
    print("\nChargement du modèle...")
    model = load_model('mon_cnn.h5')

    # 2. Créer le dataset de test
    print("Création du dataset de test...")
    (_, _, test_dataset, _, _,
     _, _, X_test, _, _, y_test, classes) = data_pipeline()

    # Calculer le nombre d'exemples et d'étapes
    num_test_examples = len(X_test)
    test_steps = (num_test_examples + BATCH_SIZE - 1) // BATCH_SIZE  # Arrondi au supérieur

    # 3. Évaluer le modèle
    print("Évaluation du modèle...")
    test_loss, test_accuracy, test_recall = model.evaluate(
        test_dataset,
        steps=test_steps,
        verbose=1
    )

    print(f"\nPerformances avec le seuil par défaut (0.5):")
    print(f"Loss: {test_loss:.4f}")
    print(f"Accuracy: {test_accuracy:.4f}")
    print(f"Recall: {test_recall:.4f}")

    # 4. Générer les prédictions
    print("\nGénération des prédictions...")

    # Collecter toutes les prédictions
    y_true = []
    y_pred_proba = []

    for images, labels in test_dataset:
        batch_predictions = model.predict(images, verbose=0)
        y_true.extend(labels.numpy())
        y_pred_proba.extend(batch_predictions.flatten())

    # Assurer que nous avons le bon nombre d'exemples
    y_true = y_true[:num_test_examples]
    y_pred_proba = y_pred_proba[:num_test_examples]

    # 5. Tracer la courbe ROC
    print("Tracé de la courbe ROC...")
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    # Trouver le seuil optimal standard (indice de Youden)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # Trouver le seuil qui garantit un rappel minimum (minimiser les faux négatifs)
    recall_idx = np.where(tpr >= min_recall)[0]
    if len(recall_idx) > 0:
        # Parmi les seuils avec un rappel suffisant, choisir celui avec la meilleure précision
        recall_thresholds = thresholds[recall_idx]
        recall_fprs = fpr[recall_idx]

        # Choisir le seuil avec le FPR minimal (meilleure précision) parmi ceux qui respectent le rappel minimum
        custom_optimal_idx = np.argmin(recall_fprs)
        custom_optimal_threshold = recall_thresholds[custom_optimal_idx]
        custom_optimal_recall = tpr[recall_idx[custom_optimal_idx]]
        custom_optimal_fpr = recall_fprs[custom_optimal_idx]
    else:
        # Si aucun seuil n'atteint le rappel minimum, prendre le seuil avec le meilleur rappel
        custom_optimal_idx = np.argmax(tpr)
        custom_optimal_threshold = thresholds[custom_optimal_idx]
        custom_optimal_recall = tpr[custom_optimal_idx]
        custom_optimal_fpr = fpr[custom_optimal_idx]

    # Créer une nouvelle figure
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Courbe ROC (AUC = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

    # Marquer les seuils optimaux
    plt.plot(fpr[optimal_idx], tpr[optimal_idx], 'ro', markersize=8,
             label=f'Seuil Youden = {optimal_threshold:.2f} (TPR={tpr[optimal_idx]:.2f}, FPR={fpr[optimal_idx]:.2f})')

    plt.plot(custom_optimal_fpr, custom_optimal_recall, 'go', markersize=8,
             label=f'Seuil Custom = {custom_optimal_threshold:.2f} (TPR={custom_optimal_recall:.2f}, FPR={custom_optimal_fpr:.2f})')

    # Ligne horizontale indiquant le rappel minimum
    plt.axhline(y=min_recall, color='r', linestyle='--', alpha=0.3, label=f'Rappel minimum = {min_recall}')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taux de faux positifs')
    plt.ylabel('Taux de vrais positifs')
    plt.title('Courbe ROC avec seuils optimaux')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)

    plt.savefig('roc_curve_thresholds.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 6. Tracer la courbe Précision-Rappel
    print("Tracé de la courbe Précision-Rappel...")
    precision, recall, pr_thresholds = precision_recall_curve(y_true, y_pred_proba)
    pr_auc = auc(recall, precision)

    plt.figure(figsize=(10, 8))
    plt.plot(recall, precision, color='darkgreen', lw=2, label=f'Courbe PR (AUC = {pr_auc:.3f})')

    # Trouver les indices correspondant aux seuils optimaux
    # Convertir les seuils ROC en indices de précision-rappel (approximativement)
    if len(pr_thresholds) > 0:
        # Trouver les indices de précision-rappel les plus proches des seuils optimaux
        optimal_pr_idx = np.abs(pr_thresholds - optimal_threshold).argmin() if len(pr_thresholds) > 0 else 0
        custom_optimal_pr_idx = np.abs(pr_thresholds - custom_optimal_threshold).argmin() if len(pr_thresholds) > 0 else 0

        # Marquer les points sur la courbe précision-rappel
        plt.plot(recall[optimal_pr_idx], precision[optimal_pr_idx], 'ro', markersize=8,
                label=f'Seuil Youden = {optimal_threshold:.2f} (P={precision[optimal_pr_idx]:.2f}, R={recall[optimal_pr_idx]:.2f})')

        plt.plot(recall[custom_optimal_pr_idx], precision[custom_optimal_pr_idx], 'go', markersize=8,
                label=f'Seuil Custom = {custom_optimal_threshold:.2f} (P={precision[custom_optimal_pr_idx]:.2f}, R={recall[custom_optimal_pr_idx]:.2f})')

    # Ligne verticale indiquant le rappel minimum
    plt.axvline(x=min_recall, color='r', linestyle='--', alpha=0.3, label=f'Rappel minimum = {min_recall}')

    plt.xlabel('Rappel')
    plt.ylabel('Précision')
    plt.title('Courbe Précision-Rappel avec seuils optimaux')
    plt.legend(loc="lower left")
    plt.grid(True, alpha=0.3)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])

    plt.savefig('precision_recall_curve_thresholds.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 8. Tracer les matrices de confusion pour les différents seuils
    print("Tracé des matrices de confusion...")

    # Seuil par défaut (0.5)
    y_pred_default = (np.array(y_pred_proba) >= 0.5).astype(int)
    cm_default = confusion_matrix(y_true, y_pred_default)

    # Seuil optimal Youden
    y_pred_optimal = (np.array(y_pred_proba) >= optimal_threshold).astype(int)
    cm_optimal = confusion_matrix(y_true, y_pred_optimal)

    # Seuil optimal personnalisé
    y_pred_custom = (np.array(y_pred_proba) >= custom_optimal_threshold).astype(int)
    cm_custom = confusion_matrix(y_true, y_pred_custom)

    # Fonction pour afficher une matrice de confusion
    def plot_cm(cm, title, filename):
        plt.figure(figsize=(8, 6))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        # Ajouter les valeurs dans les cellules
        thresh = cm.max() / 2
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                plt.text(j, i, format(cm[i, j], 'd'),
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('Vraie classe')
        plt.xlabel('Classe prédite')
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()

    # Afficher les trois matrices de confusion
    plot_cm(cm_default, f'Matrice de confusion (seuil = 0.5)', 'confusion_matrix_default.png')
    plot_cm(cm_optimal, f'Matrice de confusion Youden (seuil = {optimal_threshold:.2f})', 'confusion_matrix_optimal.png')
    plot_cm(cm_custom, f'Matrice de confusion Custom (seuil = {custom_optimal_threshold:.2f})', 'confusion_matrix_custom.png')

    # Calculer et afficher les métriques pour chaque seuil
    metrics = []
    for threshold_name, threshold_value, y_pred in [
        ("Défaut (0.5)", 0.5, y_pred_default),
        (f"Youden ({optimal_threshold:.4f})", optimal_threshold, y_pred_optimal),
        (f"Custom ({custom_optimal_threshold:.4f})", custom_optimal_threshold, y_pred_custom)
    ]:
        acc = accuracy_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        # Calculer les faux négatifs (FN) et faux positifs (FP)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

        metrics.append({
            "Seuil": threshold_name,
            "Valeur": threshold_value,
            "Accuracy": acc,
            "Recall": rec,
            "Precision": prec,
            "F1 Score": f1,
            "VP": tp,
            "FP": fp,
            "FN": fn,
            "VN": tn
        })

    # Afficher le tableau comparatif des seuils
    print("\nComparaison des différents seuils:")
    print("="*100)
    print(f"{'Seuil':20} | {'Valeur':8} | {'Accuracy':8} | {'Recall':8} | {'Precision':8} | {'F1 Score':8} | {'VP':6} | {'FP':6} | {'FN':6} | {'VN':6}")
    print("-"*100)
    for m in metrics:
        print(f"{m['Seuil']:20} | {m['Valeur']:.4f} | {m['Accuracy']:.4f} | {m['Recall']:.4f} | {m['Precision']:.4f} | {m['F1 Score']:.4f} | {m['VP']:6d} | {m['FP']:6d} | {m['FN']:6d} | {m['VN']:6d}")

    # Recommandation de seuil
    print("\nRecommandation de seuil:")
    if custom_optimal_recall >= min_recall:
        print(f"Le seuil personnalisé ({custom_optimal_threshold:.4f}) atteint un rappel de {custom_optimal_recall:.4f} ≥ {min_recall:.2f} (minimum souhaité)")
        print(f"Ce seuil minimise les faux négatifs tout en préservant une précision de {precision[custom_optimal_pr_idx]:.4f}")
        recommended_threshold = custom_optimal_threshold
    else:
        print(f"Aucun seuil n'a pu atteindre le rappel minimum de {min_recall:.2f}. Le rappel maximum est de {custom_optimal_recall:.4f}")
        recommended_threshold = custom_optimal_threshold

    # Comparer avec le seuil de Youden
    print(f"\nLe seuil de Youden ({optimal_threshold:.4f}) offre un meilleur équilibre global entre vrais positifs et faux positifs")
    print(f"Mais il produit {metrics[1]['FN']} faux négatifs contre {metrics[2]['FN']} pour le seuil personnalisé.")

    print("\nRecommandation finale:")
    if metrics[2]['FN'] < metrics[1]['FN'] and metrics[2]['Accuracy'] > 0.75:  # Si le seuil personnalisé a moins de FN et une accuracy acceptable
        print(f"Utilisez le seuil personnalisé ({custom_optimal_threshold:.4f}) pour minimiser les faux négatifs tout en maintenant des performances acceptables.")
    else:
        print(f"Le seuil de Youden ({optimal_threshold:.4f}) offre un meilleur équilibre global. Utilisez-le si minimiser les faux positifs est aussi important.")

    return roc_auc, pr_auc, optimal_threshold, custom_optimal_threshold

# --------------------------------------------------
# Étape 9: Pipeline complet
# --------------------------------------------------
def run_complete_pipeline(retrain=True, min_recall=0.90):
    """
    Exécute le pipeline complet d'entraînement et d'évaluation.

    Args:
        retrain: Booléen pour déterminer si le modèle doit être réentraîné
        min_recall: Rappel minimum souhaité pour le seuil personnalisé

    Returns:
        tuple: (model, roc_auc, pr_auc, optimal_threshold, custom_threshold)
    """
    if retrain:
        # Entraîner le modèle
        model, _, _ = create_and_train_model()
    else:
        # Charger le modèle existant
        try:
            model = load_model(MODEL_SAVE_PATH)
            print(f"Modèle chargé depuis {MODEL_SAVE_PATH}")
        except:
            print(f"Impossible de charger le modèle depuis {MODEL_SAVE_PATH}. Entraînement d'un nouveau modèle.")
            model, _, _ = create_and_train_model()

    # Évaluer le modèle avec ROC
    roc_auc, pr_auc, optimal_threshold, custom_threshold = generate_roc_curve_evaluation(min_recall)

    print("\n" + "="*50)
    print("Pipeline complet exécuté avec succès!")
    print(f"Modèle enregistré sous: {MODEL_SAVE_PATH}")
    print(f"Performances: AUC-ROC = {roc_auc:.4f}, AUC-PR = {pr_auc:.4f}")
    print(f"Seuil Youden: {optimal_threshold:.4f}")
    print(f"Seuil personnalisé: {custom_threshold:.4f}")

    return model, roc_auc, pr_auc, optimal_threshold, custom_threshold

# --------------------------------------------------
# Exécution du code
# --------------------------------------------------
if __name__ == "__main__":
    # Pour exécuter le pipeline complet avec réentraînement
    # model, roc_auc, pr_auc, optimal_threshold, custom_threshold = run_complete_pipeline(retrain=True, min_recall=0.95)

    # Pour charger un modèle existant et l'évaluer sans réentraînement
    model, roc_auc, pr_auc, optimal_threshold, custom_threshold = run_complete_pipeline(retrain=False, min_recall=0.95)

Impossible de charger le modèle depuis mon_cnn.h5. Entraînement d'un nouveau modèle.

Chargement et division des données...

Structure détectée:
- bad: 285 images
- good: 867 images

Taille des datasets:
- Entraînement: 806 images
- Validation: 173 images
- Test: 173 images

=== Avant augmentation ===

Analyse de Train original:
- Classe 0: 199 (24.7%)
- Classe 1: 607 (75.3%)

=== Application de l'augmentation ===

Configuration de l'entraînement:
- Échantillons augmentés: 408
- Nombre total d'échantillons: 1214
- Steps per epoch: 37
- Validation steps: 5


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

...

37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.8310 - loss: 0.3903 - recall: 0.8954 - val_accuracy: 0.7750 - val_loss: 0.5090 - val_recall: 0.9083 - learning_rate: 0.0020
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.8475 - loss: 0.3563 - recall: 0.9013 - val_accuracy: 0.7688 - val_loss: 0.5109 - val_recall: 0.9417 - learning_rate: 0.0020
Epoch 9/50
31/37 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.8517 - loss: 0.3231 - recall: 0.9194

37/37 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - accuracy: 0.8514 - loss: 0.3253 - recall: 0.9204 - val_accuracy: 0.7937 - val_loss: 0.5004 - val_recall: 0.9333 - learning_rate: 0.0020
Epoch 10/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.8600 - loss: 0.3355 - recall: 0.9310 - val_accuracy: 0.7750 - val_loss: 0.5108 - val_recall: 0.9417 - learning_rate: 0.0020
Epoch 11/50
31/37 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8530 - loss: 0.3307 - recall: 0.9121

37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 315ms/step - accuracy: 0.8544 - loss: 0.3303 - recall: 0.9149 - val_accuracy: 0.7937 - val_loss: 0.4951 - val_recall: 0.9333 - learning_rate: 0.0020
Epoch 12/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 306ms/step - accuracy: 0.8924 - loss: 0.2974 - recall: 0.9440 - val_accuracy: 0.7937 - val_loss: 0.5230 - val_recall: 0.9000 - learning_rate: 0.0020
Epoch 13/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - accuracy: 0.8748 - loss: 0.2819 - recall: 0.9452 - val_accuracy: 0.7437 - val_loss: 0.5401 - val_recall: 0.8000 - learning_rate: 0.0020
Epoch 14/50
31/37 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8863 - loss: 0.2718 - recall: 0.9190
Epoch 14: ReduceLROnPlateau reducing learning rate to 0.0010000000474974513.
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.8864 - loss: 0.2727 - recall: 0.9220 - val_accuracy: 0.7937 - val_loss: 0.5113 - val_recall: 0.9167 - learning_rate: 0.0020
Epoch 15/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - accuracy: 0.8805 - loss

Création du dataset de test...

Chargement et division des données...

Structure détectée:
- bad: 285 images
- good: 867 images

Taille des datasets:
- Entraînement: 806 images
- Validation: 173 images
- Test: 173 images

=== Avant augmentation ===

Analyse de Train original:
- Classe 0: 199 (24.7%)
- Classe 1: 607 (75.3%)

=== Application de l'augmentation ===

Configuration de l'entraînement:
- Échantillons augmentés: 408
- Nombre total d'échantillons: 1214
- Steps per epoch: 37
- Validation steps: 5
Évaluation du modèle...
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 326ms/step - accuracy: 0.8387 - loss: 0.4069 - recall: 0.9770

Performances avec le seuil par défaut (0.5):
Loss: 0.4419
Accuracy: 0.8150
Recall: 0.9692

Génération des prédictions...


Tracé de la courbe ROC...
Tracé de la courbe Précision-Rappel...
Tracé des matrices de confusion...

Comparaison des différents seuils:
Seuil                | Valeur   | Accuracy | Recall   | Precision | F1 Score | VP     | FP     | FN     | VN    
----------------------------------------------------------------------------------------------------
Défaut (0.5)         | 0.5000 | 0.8150 | 0.9692 | 0.8182 | 0.8873 |    126 |     28 |      4 |     15
Youden (0.7574)      | 0.7574 | 0.7919 | 0.8231 | 0.8917 | 0.8560 |    107 |     13 |     23 |     30
Custom (0.6463)      | 0.6463 | 0.8555 | 0.9538 | 0.8671 | 0.9084 |    124 |     19 |      6 |     24

Recommandation de seuil:
Le seuil personnalisé (0.6463) atteint un rappel de 0.9538 ≥ 0.95 (minimum souhaité)
Ce seuil minimise les faux négatifs tout en préservant une précision de 0.8671

Le seuil de Youden (0.7574) offre un meilleur équilibre global entre vrais positifs et faux positifs
Mais il produit 23 faux négatifs contre 6 pour le se